In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import hopsworks

project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME, api_key_value=config.HOPSWORKS_API_KEY
)

feature_store = project.get_feature_store()
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION
)

# Create a feature view if it doesn't already exist
try:
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all(),
    )
    print(f"Feature view '{config.FEATURE_VIEW_NAME}' (version {config.FEATURE_VIEW_VERSION}) created successfully.")
except Exception as e:
    print(f"Error creating feature view: {e}")

# Retrieve the feature view
try:
    feature_view = feature_store.get_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
    )
    print(f"Feature view '{config.FEATURE_VIEW_NAME}' (version {config.FEATURE_VIEW_VERSION}) retrieved successfully.")
except Exception as e:
    print(f"Error retrieving feature view: {e}")

ts_data, _ = feature_view.training_data(
    description="Time-series hourly taxi rides"
)

ts_data = ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

c:\Users\nolan\anaconda3\envs\sp25_taxi\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-03-01 19:23:49,194 INFO: Initializing external client
2025-03-01 19:23:49,197 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-01 19:23:50,814 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Error creating feature view: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1212635/featurestores/1200268/featureview). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":270179,"usrMsg":"Feature view: time_series_hourly_feature_view, version: 1","errorMsg":"The provided feature view name and version already exists"}', error code: 270179, error msg: The provided feature view name and version already exists, user msg: Feature view: time_series_hourly_feature_view, version: 1
Feature view 'time_series_hourly_feature_view' (version 1) retrieved successfully.
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (22.48s) 


In [4]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00+00:00,2,0
1,2023-01-01 01:00:00+00:00,2,0
2,2023-01-01 02:00:00+00:00,2,0
3,2023-01-01 03:00:00+00:00,2,0
4,2023-01-01 04:00:00+00:00,2,0
...,...,...,...
4926347,2025-03-01 19:00:00+00:00,263,170
4926348,2025-03-01 20:00:00+00:00,263,140
4926349,2025-03-01 21:00:00+00:00,263,154
4926350,2025-03-01 22:00:00+00:00,263,137


In [5]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4926352 entries, 0 to 4926351
Data columns (total 3 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   pickup_hour         object
 1   pickup_location_id  int32 
 2   rides               int32 
dtypes: int32(2), object(1)
memory usage: 75.2+ MB


In [6]:
import pandas as pd

ts_data["pickup_hour"] = pd.to_datetime(ts_data["pickup_hour"], errors="coerce")
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)  # Remove timezone

In [7]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,2,0
1,2023-01-01 01:00:00,2,0
2,2023-01-01 02:00:00,2,0
3,2023-01-01 03:00:00,2,0
4,2023-01-01 04:00:00,2,0
...,...,...,...
4926347,2025-03-01 19:00:00,263,170
4926348,2025-03-01 20:00:00,263,140
4926349,2025-03-01 21:00:00,263,154
4926350,2025-03-01 22:00:00,263,137


In [8]:
filtered_ts_data = ts_data[ts_data["pickup_hour"] > "2025-02-01"]

In [ ]:
from pmdarima import auto_arima
import time
from sklearn.metrics import mean_absolute_error
import warnings

warnings.filterwarnings(action='ignore', category=FutureWarning)

locations = filtered_ts_data["pickup_location_id"].unique()

models = {}
model_maes = {}
for location in locations:
    t_start = time.time()
    location_data = filtered_ts_data[filtered_ts_data["pickup_location_id"] == location]

    train_size = int(len(location_data) * 0.8)
    train, test = location_data['rides'][:train_size], location_data["rides"][train_size:]
    print(f"Starting to train model for location {location}...")
    model = auto_arima(train,
                       start_P=1, start_q=1, max_p=5,
                       seasonal=True,
                       stepwise=True,
                       m=24)
    models[location] = model
    t_end = time.time()
    print(f"ARIMA model for location {location} has been created! Time: {t_end - t_start}")

    predictions = model.predict(n_periods = int(len(test)))
    mae = mean_absolute_error(test, predictions)
    model_maes[location] = mae
    print(f"Test MAE for location {location} was {mae}")

Starting to train model for location 2...
ARIMA model for location 2 has been created! Time: 4.807953596115112
Test MAE for location 2 was 0.003597122266934744
Starting to train model for location 3...


ARIMA model for location 3 has been created! Time: 22.07348132133484
Test MAE for location 3 was 0.26037909041225793
Starting to train model for location 4...


ARIMA model for location 4 has been created! Time: 31.811100959777832
Test MAE for location 4 was 6.091626112616581
Starting to train model for location 6...


ARIMA model for location 6 has been created! Time: 111.79216980934143
Test MAE for location 6 was 0.06546346662608325
Starting to train model for location 7...


ARIMA model for location 7 has been created! Time: 61.38252520561218
Test MAE for location 7 was 1.5131930610748485
Starting to train model for location 8...


...

ARIMA model for location 155 has been created! Time: 135.7608859539032
Test MAE for location 155 was 0.5800669619335724
Starting to train model for location 156...
ARIMA model for location 156 has been created! Time: 0.07958006858825684
Test MAE for location 156 was 0.0
Starting to train model for location 157...


ARIMA model for location 157 has been created! Time: 17.936073780059814
Test MAE for location 157 was 0.7962871913155014
Starting to train model for location 158...


ARIMA model for location 158 has been created! Time: 53.294631481170654
Test MAE for location 158 was 26.04959794527705
Starting to train model for location 159...


ARIMA model for location 159 has been created! Time: 126.78769898414612
Test MAE for location 159 was 0.6440438610457958
Starting to train model for location 160...


ARIMA model for location 160 has been created! Time: 16.760437965393066
Test MAE for location 160 was 0.2692696776578132
Starting to train model for location 161...
